In [ ]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

# Recall that the default shuffle partitions is 200.  We want to bring that down to a reasonable size for both our data and our Spark cluster
# 4 is reasonable for a free Colab 
spark.conf.set("spark.sql.shuffle.partitions", 4)

In [ ]:
# Read in data from S3 Bucket

# Create a lookup table for the 500 cities. 


# Create a lookup table for the airport codes. 



In [ ]:
# Look over the delayed flight data.


In [ ]:
# Look over the data of the 500 cities.


In [ ]:
# Look over the airport codes.


In [ ]:
# Filter the airport codes to only contain rows whose `country` equals `USA`


In [ ]:
# Filter the latitude and longitude dataframe to only contain the 'name','latitude','longitude','admin1_code' fields and rows whose `country_code` equals `US`


In [ ]:
# Create temporary views for each of our DataFrames



In [ ]:
# First, join the airport codes lookup table to the delayed flight DataFrame 
# and add the city of origin and destination like we did in the instructor demo.  

start_time = time.time()

spark.sql("""

""").show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Add the `origin_latitude` and `origin_longitude` fields by joining the `lookup_geo` view 
# to the `lookup_city` view and the delayed flight DataFrame.
# Note:  The two lookup views do not have matching columns, so we must be mindful what names are used when joining both views together.

start_time = time.time()

spark.sql("""

""").show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Finally, add the `dest_latitude` and `dest_longitude` fields by joining the `lookup_geo` view again as another alias, `geo_dest`.

start_time = time.time()

spark.sql("""

""").show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Run the same query with a Broadcast hint for either table
 
start_time = time.time()

spark.sql("""
select /*+ BROADCAST() */ 

""").show()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Write a sql (we used a CTE here) that does some aggregations on the new data.  The purpose of this SQL is to add some processing time.
# Note the runtime
start_time = time.time()

spark.sql("""
with allColumns
(select /*+ BROADCAST() */ 

""").show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Cache your largest temporary view
# Note: when we use SparkSQL to cache a table, the table is immediately cached (no lazy evaluation), when using Pyspark it will not be cached until an action is ran.


In [ ]:
# Check that your table is cached 
spark.catalog.isCached()

In [ ]:
# Run the same query again with the data cached. This should greatly improve the run time.  
# Keep in mind we are not working with particularly large data here so the improvements may not be dramatic.

start_time = time.time()

spark.sql("""
with allColumns
(select /*+ BROADCAST() */ 

""").show()

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# you can even cache a large lookup table.
spark.sql()

In [ ]:
# Run the same query again with the data cached. This should greatly improve the run time.

start_time = time.time()

spark.sql("""
with allColumns
(select /*+ BROADCAST() */ 

""").show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Remember to uncache the table as soon as you are done.
spark.sql()
spark.sql()

In [ ]:
# Verify that the table is no longer cached
if spark.catalog.isCached() or spark.catalog.isCached():
  print("a table is till cached")
else:
  print("all clear")